In [1]:
%reset_selective -f b

# Este comando le indica a Python que las figuras se deben generar dentro de la misma Notebook, no en una ventana
## Solo para notebooks.
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import colors
#from sklearn.svm import SVC
import numpy as np
import glob
import pyart
import scipy.io as sio
import re



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119 


In [31]:
def read_files(day,init_time):
    day = str(day)
    init_time = str(init_time)
    path_user_radar = './febdBZ/'+ day + '/'
    FileList_radar = np.sort(glob.glob(path_user_radar+'*.nc'))
    
    file2read_radar=0
    for i in range(len(FileList_radar)):
        file2read_radar_time = FileList_radar[i].split('_')[1][:4]
        if (file2read_radar_time == init_time):
            file2read_radar = FileList_radar[i]
    
    return(file2read_radar)
 



def chequear_existencia(day,init_time):
    day = str(day)
    init_time = str(init_time)
    path_user_radar = './febdBZ/'+ day + '/'
    FileList_radar = np.sort(glob.glob(path_user_radar+'*.nc'))
    existencia = False
    for i in range(len(FileList_radar)):
        file2read_radar_time = FileList_radar[i].split('_')[1][:4]
        if (file2read_radar_time == init_time):
            existencia=True
    if not existencia:
        print('No se encontró el archivo',day, init_time)
    return(existencia)

def get_volumes(day, init_time, bins=240,diff=30):
    diff=diff/10
    day = str(day)
    init_time = str(init_time)
    path_user_radar = './febdBZ/'+ day + '/'
    FileList_radar = np.sort(glob.glob(path_user_radar+'*.nc'))
    proof = 0
    file2read_radar=0
    
    for i in range(len(FileList_radar)):
        file2read_radar_time = FileList_radar[i].split('_')[1][:4]
        if (file2read_radar_time == init_time):
            file2read_radar = FileList_radar[i]
            proof= int(i+diff)
           
    
    file2follow = FileList_radar[proof]   
               

    radar = pyart.io.read(file2read_radar)
    radar2 = pyart.io.read(file2follow)
    
    gatefilter = pyart.filters.GateFilter(radar)
    
    gatefilter.exclude_transition()
    gatefilter.exclude_masked('dBZ')
    
    grid = pyart.map.grid_from_radars((radar,), 
                                gatefilters=(gatefilter, ),
                                grid_shape=(1, bins, bins),
                                grid_limits=((0, 0), (-bins*1000, bins*1000), (-bins*1000, bins*1000)),
                                fields=['dBZ'])
    
    gatefilter2 = pyart.filters.GateFilter(radar2)
    
    gatefilter2.exclude_transition()
    gatefilter2.exclude_masked('dBZ')
    
    grid2 = pyart.map.grid_from_radars(
        (radar2,), gatefilters=(gatefilter2, ),
        grid_shape=(1, bins, bins),
        grid_limits=((0, 0), (-bins*1000, bins*1000), (-bins*1000, bins*1000)),
        fields=['dBZ'])
            
            
    
    return (grid,grid2)

def get_labels(lines,threshold=35):
    samples = []
    eventos = len(lines)
    for i in range(eventos):
        (fecha,hora)=(lines[i].strip().split("\t")[0],lines[i].strip().split("\t")[1])
        if chequear_existencia(fecha,hora):
            samples.append(get_volumes(fecha,hora)[1])
        else:
            eventos=eventos-1
    
    labels = []
    for i in range(eventos):
        grid = samples[i].fields['dBZ']['data'][0]
        for x in range(240):
            for y in range(240):
                if not grid.mask[x,y]:
                    if grid[x,y]<35:
                        grid.data[x,y]=0
                    else:
                        grid.data[x,y]=1
        labels.append(grid)
    return(labels)

def get_rays(day):
    folder=str(day)[:6]
    dia=str(day)
    path_user_rayos = './rayos/'+ folder + '/' + dia
    # Con esto levanto todos los datos de rayos país ese día
    with open(path_user_rayos+'.txt') as f:
        lights = f.readlines()
    return(lights)

def get_limits(day,init_time):
    grid=get_volumes(day,init_time)[0]
    lat_grid = grid.get_point_longitude_latitude(level=0, edges=False)[1]
    lon_grid = grid.get_point_longitude_latitude(level=0, edges=False)[0]
    extLat   = [np.amin(lat_grid),np.amax(lat_grid)]
    extLon   = [np.amin(lon_grid),np.amax(lon_grid)]
    return(extLon,extLat)

def rayos_correctos(archivo,limite_tiempo,limite_lon,limite_lat):
    index = []
    for i in range(len(archivo)):
        foo=archivo[i].split()
        t=(foo[3]+foo[4]+foo[5])
        if (t<limite_tiempo[1]) and (t>limite_tiempo[0]) :
            
            if(float(foo[7])>limite_lat[0]) and (float(foo[7])<limite_lat[1]):
                
                if(float(foo[8])>limite_lon[0]) and (float(foo[8])<limite_lon[1]):
                    index.append(i)
    return(index)
    

    

In [3]:
with open('dates.txt') as f:
    datos_samples = f.readlines()


In [10]:
etiquetas = get_labels(datos_samples)


('No se encontr\xc3\xb3 el archivo', '20170219', '2240')


In [39]:
type(etiquetas[0])

numpy.ma.core.MaskedArray

### Rayos


In [4]:
(fecha,hora)=(datos_samples[0].strip().split("\t")[0],datos_samples[0].strip().split("\t")[1])
(Lat, Lon)= get_limits(fecha,hora)

In [5]:
print(Lat,Lon)

([-66.754157243330312, -61.225976756669681], [-38.698056318536423, -34.352495282538385])


In [44]:
bins=240
difLat   = bins/(Lat[1]-Lat[0])
difLon   = bins/(Lon[1]-Lon[0])

In [36]:
#Indices de los rayos que estan en los límites del radar
eventos = len(datos_samples)
index = []
eventos=2
for i in range(eventos):
    (fecha,hora)=(datos_samples[i].strip().split("\t")[0],datos_samples[i].strip().split("\t")[1])
    if chequear_existencia(fecha,hora):
    
        file2read_radar=read_files(fecha,hora)
        
        horario = (file2read_radar.split('_')[1],file2read_radar.split('_')[4])
        lines = get_rays(fecha)
        ind = rayos_correctos(lines,horario, Lon, Lat) 
        #print(ind)
                           
    else:
        eventos=eventos-1
    index.append(ind)
    
#Ahora solo necesito mapearlos en una grilla


In [46]:

indices = np.zeros((bins,bins), dtype=object) # En este array tengo el índice del rayo en la lista
time = np.zeros((bins,bins), dtype=object) # En este array guardo el tiempo en el que ocurrió cada rayo
intensidad = np.zeros((bins,bins), dtype=object) # Y en este la intensidad
FR = np.zeros((bins,bins), dtype=object) # en este array calculo la cantidad de rayos y lo divido por el intervalo de ocurrencia
i=0
j=0

for l in range(len(Rayos['x'])):
    i=Rayos['x'][l]
    j=Rayos['y'][l]
    indices[i,j]=np.append(indices[i,j],l)
    intensidad[i,j]=np.append(intensidad[i,j],Rayos['Int'][l])
    time[i,j]=np.append(time[i,j],Rayos['Time'][l])
    
for i in range(bins):
    for j in range(bins):
        if not isinstance(indices[i,j], int):
            indices[i,j] = np.delete(indices[i,j],0)
            intensidad[i,j] = np.delete(intensidad[i,j],0)
            time[i,j] = np.delete(time[i,j],0)
            FR[i,j] = np.size(indices[i,j])
            
            if FR[i,j]>1:
                tiempo=[]
                for ind in indices[i,j]:
                    
                    tiempo.append(Rayos['Time'][ind])
                    
                    
                dif=float(max(tiempo))-float(min(tiempo))
                if dif != 0:
                    #print(FR[i,j])
                    #print(float(max(tiempo)),float(min(tiempo)))
                    FR[i,j]=FR[i,j]/dif
                else:
                    FR[i,j]=1
            
                
                
Rayos = {'Lat': latitude, 'Long': longitude, 'Time': time, 'Int': intensity, 'x':lonBin, 'y':latBin, 
         'Ind': indices, 'FR':FR}
# Agrego los arrays que conseguí en un diccionario. Quisiera que estos sean los campos.
Campos = {'Ind': indices, 'Intensidad': intensidad, 'Tiempo':time, 'FR': FR}

('No se encontr\xc3\xb3 el archivo', '20170219', '2240')
